In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
from huggingface_hub import push_to_hub_keras, from_pretrained_keras
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Audio

# Charger le dataset Speech Commands
dataset, info = tfds.load("speech_commands", with_info=True, as_supervised=True)
train_data, test_data = dataset["train"], dataset["test"]

# Prétraitement : Transformer l’audio en spectrogramme
def preprocess(audio, label):
    audio = tf.squeeze(audio, axis=-1)  # Retirer la dernière dimension
    spectrogram = tf.signal.stft(audio, frame_length=256, frame_step=128)
    spectrogram = tf.abs(spectrogram)
    return spectrogram, label

train_data = train_data.map(preprocess).batch(32)
test_data = test_data.map(preprocess).batch(32)

# Afficher la forme d'onde du signal audio
plt.figure(figsize=(10, 4))
plt.plot(sample_audio_np)
plt.title(f"Exemple de signal audio - Label: {sample_label.numpy()}")
plt.xlabel("Temps (échantillons)")
plt.ylabel("Amplitude")
plt.show()

# Écouter l’audio
Audio(sample_audio_np, rate=16000)

# Construire le modèle RNN
model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(128, activation="relu", return_sequences=True),
    tf.keras.layers.SimpleRNN(64, activation="relu"),
    tf.keras.layers.Dense(35, activation="softmax")  # 35 classes
])


model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Entraînement
model.fit(train_data, epochs=5, validation_data=test_data)

# Sauvegarde et upload du modèle sur Hugging Face
model.save("rnn_speech_commands")
push_to_hub_keras(model, "rnn_speech_commands")

# 📥 **Charger le modèle sauvegardé depuis Hugging Face**
loaded_model = from_pretrained_keras("rnn_speech_commands")

# 📊 **Faire une prédiction sur un nouvel échantillon**
sample_spectrogram, _ = preprocess(sample_audio, sample_label)
sample_spectrogram = tf.expand_dims(sample_spectrogram, axis=0)  # Ajouter une dimension batch

predictions = loaded_model.predict(sample_spectrogram)
predicted_label = np.argmax(predictions)

# Afficher le résultat
print(f"🔍 Prédiction du modèle : Classe {predicted_label}")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

KeyboardInterrupt: 